# Numerical Gradient Checking

We would highly recommend looking at `neural_networks.grad_check.check_gradients` and making sure you understand how numerical gradient checking is being carried out. This function is used in the notebook to check the gradients of the neural network layers you write. Make sure to check the gradient of a layer after finishing its implementation.

The function returns the relative error of the numerical gradient (approximated using finite differences) with respect to the analytical gradient (computed via backpropagation). Correct implementations should get very small errors, usually less than `1e-8` for 64-bit float matrices (the default).

In [42]:
%load_ext autoreload
%autoreload 2

import numpy as np
from neural_networks.utils import check_gradients
from neural_networks.layers import FullyConnected, Conv2D, Pool2D
from neural_networks.activations import Linear, Sigmoid, TanH, ReLU, SoftMax

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Gradient Checks for Activation Functions

### Linear Activation

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 3)

# initialize a fully connected layer
# and perform a forward and backward pass
linear_activation = Linear()
_ = linear_activation.forward(X)
grad = linear_activation.backward(X, dLdY)

# check the gradients w.r.t. each parameter
print(
    f"Relative error for linear activation:",
    check_gradients(
        fn=linear_activation.forward,  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=X,        # the variable w.r.t. which we are taking the gradient
        dLdf=dLdY,  # gradient at previous layer
    )
)

### Sigmoid Activation

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 3)

# initialize a fully connected layer
# and perform a forward and backward pass
sigmoid_activation = Sigmoid()
_ = sigmoid_activation.forward(X)
grad = sigmoid_activation.backward(X, dLdY)

# check the gradients w.r.t. each parameter
print(
    f"Relative error for sigmoid activation:",
    check_gradients(
        fn=sigmoid_activation.forward,  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=X,        # the variable w.r.t. which we are taking the gradient
        dLdf=dLdY,  # gradient at previous layer
    )
)

### Tanh Activation

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 3)

# initialize a fully connected layer
# and perform a forward and backward pass
tanh_activation = TanH()
_ = tanh_activation.forward(X)
grad = tanh_activation.backward(X, dLdY)

# check the gradients w.r.t. each parameter
print(
    f"Relative error for tanh activation:",
    check_gradients(
        fn=tanh_activation.forward,  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=X,        # the variable w.r.t. which we are taking the gradient
        dLdf=dLdY,  # gradient at previous layer
    )
)

### ReLU Activation

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 3)

# initialize a fully connected layer
# and perform a forward and backward pass
relu_activation = ReLU()
out = relu_activation.forward(X)
grad = relu_activation.backward(X, dLdY)

# check the gradients w.r.t. each parameter
print(
    f"Relative error for relu activation:",
    check_gradients(
        fn=relu_activation.forward,  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=X,        # the variable w.r.t. which we are taking the gradient
        dLdf=dLdY,  # gradient at previous layer
    )
)

### Softmax Activation

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 3)

# initialize a fully connected layer
# and perform a forward and backward pass
softmax_activation = SoftMax()
_ = softmax_activation.forward(X)
grad = softmax_activation.backward(X, dLdY)

# check the gradients w.r.t. each parameter
print(
    f"Relative error for softmax activation:",
    check_gradients(
        fn=softmax_activation.forward,  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=X,        # the variable w.r.t. which we are taking the gradient
        dLdf=dLdY,  # gradient at previous layer
    )
)

## Gradient Checks for Full Layers (Linear Activations)

### Fully Connected Layer

In [ ]:
X = np.random.randn(2, 3)
dLdY = np.random.randn(2, 4)

# initialize a fully connected layer
# and perform a forward and backward pass
fc_layer = FullyConnected(n_out=4, activation="linear")
_ = fc_layer.forward(X)
_ = fc_layer.backward(dLdY)

# check the gradients w.r.t. each parameter
for param in fc_layer.parameters:
    print(
        f"Relative error for {param}:",
        check_gradients(
            fn=fc_layer.forward_with_param(param, X),  # the function we are checking
            grad=fc_layer.gradients[param],  # the analytically computed gradient
            x=fc_layer.parameters[param],  # the variable w.r.t. which we are taking the gradient
            dLdf=dLdY,                     # gradient at previous layer
        )
    )

### Conv Layer

In [11]:
testX = np.array(
    [ 
        [#B0
            [ #h0
                [ #w0
                    2,0
                ],
                [ #w1
                    2,0
                ]
            ],
            [ #h1
                [ #w0
                    2,0
                ],
                [ #w1
                    2,0
                ]
            ],
        ],

        [#B1
            [ #h0
                [ #w0
                    0,1
                ],
                [ #w1
                    0,1
                ]
            ],
            [ #h1
                [ #w0
                    0,1
                ],
                [ #w1
                    0,1
                ]
            ],
        ],   

    ]
)

filter = np.array(
    [ 
       [#h0
           [#w0
               [#cin0
                   1, 0
               ],
               [#cin1
                   2, 0
               ]
           ],
           [#w1
               [#cin0
                   1, 0
               ],
               [#cin1
                   2, 0
               ]
           ]
       ],

       [#h1
           [#w0
               [#cin0
                   1, 0
               ],
               [#cin1
                   2, 0
               ]
           ],
           [#w1
               [#cin0
                   1, 0
               ],
               [#cin1
                   2, 1
               ]
           ]
       ]

    ]
)

print(testX.shape)
# print(testX)
print(filter.shape)
# print(filter)
Z = np.zeros((2, 3, 3, 2))
b = np.array([[1,1]])
# print(b.shape)
out = np.einsum('...hwc, hwcd -> ...d', testX, filter)

# print(out)
Z[:, 0, 0, :] = out + b
# print(Z)


(2, 2, 2, 2)
(2, 2, 2, 2)


In [17]:
multiply = np.array([2, 1])
multiply.shape

out = np.einsum('bhwc, b -> hwc', testX, multiply)
print(out)

[[[4 1]
  [4 1]]

 [[4 1]
  [4 1]]]


In [45]:
X = np.random.randn(2, 6, 6, 7)
dLdY = np.random.randn(2, 6, 6, 4)

# initialize a fully connected layer
# and perform a forward and backward pass
conv_layer = Conv2D(
    n_out=4,
    kernel_shape=(3, 3),
    activation="linear",
    weight_init="uniform",
    pad="same",
)
_ = conv_layer.forward(X)
_ = conv_layer.backward(dLdY)

# check the gradients w.r.t. each parameter
for param in conv_layer.parameters:
    print(
        f"Relative error for {param}:",
        check_gradients(
            fn=conv_layer.forward_with_param(param, X),  # the function we are checking
            grad=conv_layer.gradients[param],  # the analytically computed gradient
            x=conv_layer.parameters[param],  # the variable w.r.t. which we are taking the gradient
            dLdf=dLdY,                     # gradient at previous layer
        )
    )

Relative error for W: 1.0191080717107424e-10
Relative error for b: 4.9835324999004664e-11


In [35]:
np.argmax(testX, axis = 1)

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0]]], dtype=int64)

In [33]:
X = np.random.randn(2, 5, 6, 7)
dLdY = np.random.randn(2, 5, 6, 4)

# initialize a fully connected layer
# and perform a forward and backward pass
poolLayer = Pool2D(
    kernel_shape=(3, 3),
    pad="same",
)
_ = poolLayer.forward(X)
_ = poolLayer.backward(dLdY)

# check the gradients w.r.t. each parameter
for param in conv_layer.parameters:
    print(
        f"Relative error for {param}:",
        check_gradients(
            fn=poolLayer.forward_with_param(param, X),  # the function we are checking
            grad=poolLayer.gradients[param],  # the analytically computed gradient
            x=poolLayer.parameters[param],  # the variable w.r.t. which we are taking the gradient
            dLdf=dLdY,                     # gradient at previous layer
        )
    )

(2, 1, 3, 7)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "f:\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Frank\AppData\Local\Temp\ipykernel_18080\2926503589.py", line 10, in <module>
    _ = poolLayer.forward(X)
        ^^^^^^^^^^^^^^^^^^^^
  File "f:\CS189-Projects\hw6\hw6_release\code\neural_networks\layers.py", line 475, in forward
    indices[:, rowIdx, colIdx, :] = np.argmax(window, axis=1, keepdims=True)
    ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not broadcast input array from shape (2,1,3,7) into shape (2,7)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "f:\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\Anaconda\Lib\site-packages\IPython\core\ultratb.py", l

In [ ]:
from neural_networks.losses import CrossEntropy

num_pts = 5
num_classes = 6

# one-hot encoded y
y_idxs = np.random.randint(0, num_classes, (num_pts,))
y = np.zeros((num_pts, num_classes))
y[range(num_pts), y_idxs] = 1

# normalized predictions
scores = np.random.uniform(0, 1, size=(num_pts, num_classes))
y_hat = scores / scores.sum(axis=1, keepdims=True)

cross_entropy_loss = CrossEntropy("cross_entropy")

def forward_fn(Y, Y_hat):    
    def inner_forward(Y_hat):
        return cross_entropy_loss.forward(Y, Y_hat)
    return inner_forward

loss = cross_entropy_loss.forward(y, y_hat)
grad = cross_entropy_loss.backward(y, y_hat)

print(
    f"Relative error for cross entropy loss:",
    check_gradients(
        fn=forward_fn(y, y_hat),  # the function we are checking
        grad=grad,  # the analytically computed gradient
        x=y_hat,        # the variable w.r.t. which we are taking the gradient
        dLdf=1,  # gradient at previous layer
    )
)

In [ ]:
A = np.random.rand(5, 5)
B = np.random.rand(5, 5)
#Question 1
print("Einsum Trace: ", np.einsum('ii -> ', A))
print("Np Trace: ", np.trace(A))

#Question 2
print("Einsum Matmul: ", np.linalg.norm(np.einsum('ij, jk -> ik', A, B)))
print("Np Matmul: ", np.linalg.norm(A@B))


A = np.random.rand(3, 4, 5)
B = np.random.rand(3, 5, 6)
#Question 3
print("Einsum batch matmul: ", np.linalg.norm(np.einsum('kij, kjl -> kil', A, B)))

result = np.empty((3, 4, 6))
for k in range(3):
    # print(A[k] @ B[k])
    result[k] = A[k] @ B[k]
print("Np batch matmul: ", np.linalg.norm(result))